In [ ]:
from vllm import LLM, SamplingParams

https://github.com/vllm-project/vllm/issues/13127

`vllm` does not yet support `transformers` 5.0. You can install an older version of `transformers`.
For simplicity, I used a different kernel here with an old version of the library installed.
As this notebook is just for demonstration how you can save RAM (and increase the speed),
you do not necessarily have to run it!

Unfortunately, the *monkey patch* did not work for me

In [ ]:
model_name = "Qwen/Qwen3-8B-AWQ"
llm = LLM(model=model_name, max_model_len=16384, trust_remote_code=True)

In [ ]:
!nvidia-smi

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "How many 'r's are in 'strawberry'?"}
]

In [ ]:
sampling_params = SamplingParams(
  max_tokens=1024,
  temperature=0.0,
)

In [ ]:
%%time
output = llm.chat(messages=messages, sampling_params=sampling_params)

In [ ]:
for o in output:
    prompt = o.prompt
    generated_text = o.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    print(generated_text)

In [ ]:
print(output[0].outputs[0].text)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(output[0].outputs[0].text))